In [1]:
import numpy as np
import pandas as pd
#!pip install anndata --target D:/usr/local/lib/python3.8/dist-packages
import anndata as ad
from collections import defaultdict
from rnasieve.preprocessing import model_from_raw_counts
print(ad.__version__)

0.7.8


This file uses guidance from https://github.com/theislab/anndata-tutorials/blob/master/getting-started.ipynb
 to get the jcoffman008 data as an AnnData structure so that it can be used for RNAsieve as in https://github.com/songlab-cal/rna-sieve


In [2]:
##-----loading in jcoffman007 expression and design data -----
#load in jcoffman007 count data (raw counts)
widejcoffman007_count = pd.read_csv("bulk_count_2dpa.csv.gz", compression="gzip")
#changing GeneID_genename format to just GeneID
# widejcoffman007_count["gene_id"]=widejcoffman007_count["gene_id"].str[:18]
#load in design file. note that each row is a sample that matches up with the "data" dataframe  
jcoffman007_design= pd.read_csv("bulk_design_2dpa.csv.gz", compression="gzip",index_col=0)


##-----loading in atlas expression and design data----- 
#uploading the expression data for the reference (atlas)
wideatlas_count = pd.read_csv("single_count_2dpa.csv.gz", compression="gzip")
#changing GeneID|genename format to just GeneID
# wideatlas_count["gene"]=wideatlas_count["gene"].str[:18]
#uploading the meta/"obs" for the reference (atlas)
''' 
atlas_meta_alldpf = pd.read_csv("meta.tsv", sep="\t",index_col=0)
#restricting meta to only 5dpf 
sample_names_5dpf= ['5a_5dpf', '5b_5dpf']
atlas_meta= atlas_meta_alldpf[atlas_meta_alldpf['sample_name'].isin(sample_names_5dpf)]
'''
atlas_design = pd.read_csv("single_design_2dpa.csv.gz", compression="gzip",index_col=0)


In [3]:
#altering both expression datasets to contain identical sets of genes 
jcoffman007_genes= widejcoffman007_count["genes"].tolist()
atlas_genes= wideatlas_count["genes"].tolist()
genes_in_both= list(set(jcoffman007_genes) & set(atlas_genes))
print("there are",len(genes_in_both),"genes that are in both the atlas and jcoffman007 count data") 
widejcoffman007_count=widejcoffman007_count[widejcoffman007_count["genes"].isin(genes_in_both)]
wideatlas_count=wideatlas_count[wideatlas_count["genes"].isin(genes_in_both)]
#sorting to be in same order
widejcoffman007_count=widejcoffman007_count.sort_values(by=['genes'])
wideatlas_count=wideatlas_count.sort_values(by=['genes'])

there are 24324 genes that are in both the atlas and jcoffman007 count data


In [4]:
#------making annData structure that holds the counts and meta for jcoffman007-----
#convert to having genes as columns, samples as rows (first have to make the gene ID the row label)
widejcoffman007_count= widejcoffman007_count.set_index("genes")
jcoffman007_count= widejcoffman007_count.transpose()
jcoffman007 = ad.AnnData(jcoffman007_count, obs=jcoffman007_design)

#-----making annData structure that holds the counts and meta for atlas----- 
wideatlas_count=wideatlas_count.set_index("genes")
atlas_count= wideatlas_count.transpose()
atlas = ad.AnnData(atlas_count, obs=atlas_design)

In [5]:
#modeling off of the raw counts prep section from example.ipynb from song lab github

# grouping the cells in the reference data by cluster 
print('Aggregating by cluster...')
counts_by_cluster = {}
for i in range(len(atlas)):
    sc = atlas[i]
    if len(sc.obs['major.cl']) == 0:
        continue
    cell_cluster = sc.obs['major.cl'][0]
    if cell_cluster not in counts_by_cluster:
        counts_by_cluster[cell_cluster] = np.empty((sc.X.shape[1], 0), dtype=np.float32)
    counts_by_cluster[cell_cluster] = np.hstack(
        (counts_by_cluster[cell_cluster], sc.X.toarray().reshape(-1, 1)))
    

# Bulk prep
print('Aggregating bulks by name...')
G = jcoffman007.n_vars
bulk_by_time = defaultdict(list)
for i in range(len(jcoffman007)):
    bulk = jcoffman007[i]
    if len(bulk.obs['name']) == 0:
        continue
    time = bulk.obs['name'][0]
    bulk_by_time[time].append(bulk.X.toarray().reshape(-1, 1))

bulk_labels = []
psis = np.empty((G, 0), dtype=np.float32)
for name in sorted(bulk_by_time.keys()):
    bulks = bulk_by_time[name]
    for i in range(len(bulks)):
        bulk_labels.append("{} name, subject {}".format(name, i))
        psis = np.hstack((psis, bulks[i]))
        
print('Done!')

Aggregating by cluster...
Aggregating bulks by name...
Done!


In [6]:
counts_by_cluster['Intermediate Epithelial'].shape

(24324, 2572)

In [7]:
model, cleaned_psis = model_from_raw_counts(counts_by_cluster, psis[:, :18])

In [8]:
cleaned_psis

,Bulk 0,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5
0,3169.714111,2648.003418,3220.527832,2571.190918,2599.433105,2772.922852
1,1028.000000,922.000000,997.000000,853.000000,834.000000,815.000000
2,3111.187256,2748.340088,3177.886719,3509.776855,3487.701904,2715.402588
3,173177.500000,162114.765625,183359.062500,158043.296875,128596.046875,121123.156250
4,1127.342651,1007.000000,1176.000000,1362.000000,1084.214111,1094.000000
...,...,...,...,...,...,...
6176,421.000000,348.000000,452.000000,219.000000,225.000000,234.000000
6177,7920.672363,5529.533691,7514.861816,5661.808105,6905.750977,5512.356445
6178,921.379761,827.557495,966.561523,810.771240,815.942383,760.369873
6179,2319.089600,2001.473755,2300.756104,1894.161743,1949.288330,1831.039551


In [9]:
output_table= model.predict(cleaned_psis) #this takes a minute ... 
output_table.to_csv('rnasieve_jcoffman007_2dpa.csv')

In [10]:
# In this example, the intervals at a significance level of 0.05 indicate the estimate is poor.
# We set sig=0.9999 for the sake of visualization.
model.compute_marginal_confidence_intervals(sig=0.2)

[[(0.39279904904862045, 0.4339203598266686),
  (0.013543772263789241, 0.02122437598862047),
  (0.03354331621423261, 0.0791739657618851),
  (0.1868903603033549, 0.2105979045726495),
  (0.023811115050614957, 0.044052833240915974),
  (0.25933282957994214, 0.30111011814870575)],
 [(0.39523212355206416, 0.43509272822748724),
  (0.014397424306273585, 0.021662071882900747),
  (0.04724168701029156, 0.09263505173123268),
  (0.18895122244120732, 0.21227417464374565),
  (0.027345830234615277, 0.0427759011426959),
  (0.24079961565798308, 0.28159216916950275)],
 [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)],
 [(0.3825627347521914, 0.42460667723098044),
  (0.042236935317331595, 0.059114815400906365),
  (0.0, 0.0031438711640459824),
  (0.1639263419085366, 0.18934736279739767),
  (0.02120754169009811, 0.047722006368559805),
  (0.3136193980423656, 0.3556561864916322)],
 [(0.3692754126893953, 0.41000420294547885),
  (0.035122619082586765, 0.04904958533089812),
  (0.0, 0.006322

In [11]:
model.plot_proportions('bar').properties(title="Bar visualization ")

alt.FacetChart(...)

In [12]:
model.plot_proportions('stacked').properties(title="Stacked visualization")

alt.Chart(...)